In [149]:
import spacy 
import numpy as np
import emb_spacy
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.metrics import Accuracy, Precision, Recall, F1Score, MeanSquaredError




from emb_spacy import get_embedding
from input import count_examples_and_max_length, pad_sentences_from_file
import label

### Lecture des données 

In [81]:
# Ouverture en lecture des données
with open("train_corpus", "r", encoding="utf-8") as file:
    data = file.readlines()

# Compter les exemples et trouver la taille maximale
num_examples, MAX_SEQ_SIZE = count_examples_and_max_length(data)

# Affichage des résultats
print("Nombre d'exemples :", num_examples) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE)

vec_word, sortie = pad_sentences_from_file("train_corpus", MAX_SEQ_SIZE)
print("vecteur : ", vec_word.shape)
# vec_word.shape : (3945, 41)
# sortie.shape : (3945, 41)

Nombre d'exemples : 3945
Taille maximale de la phrase : 41
vecteur :  (3945, 41)


### Création des vecteurs d'entrée et de sortie

In [82]:
# Création du vecteur d'entrée 
entree = np.zeros((num_examples, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word):
    for j, word in enumerate(sentence):
        if word:
            entree[i, j] = get_embedding(word)
        else:
            entree[i, j] = np.zeros(300)

print("entree shape : ", entree.shape)
print("sortie shape : ", sortie.shape)

entree shape :  (3945, 41, 300)
sortie shape :  (3945, 41)


In [83]:
labels = label.extract_label("atis.train")
label_one_hot_dict = {label_: label.get_vector_from_label(label_) for label_ in labels} # création d'un dictionnaire associant chaque label à son vecteur one hot 

#print("LABELS :", labels)
nbLabels = len(labels)  # Nombre d'étiquettes potentielles
embedding_size = len(entree[0][0])

tailleDictionnaire = emb_spacy.get_size_dict()  # Taille du dictionnaire de mots

In [84]:
# print(sortie[0])
sortie_one_hot = np.zeros((sortie.shape[0], sortie.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(sortie.shape[0]):
    for j in range(sortie.shape[1]):
        label = sortie[i, j]
        if label == '0':
            sortie_one_hot[i, j] = zero_vec
        else:
            sortie_one_hot[i, j] = label_one_hot_dict[label]

In [85]:
np.save('sortie_one_hot.npy', sortie_one_hot)

In [86]:
sortie[0]

array(['O', 'O', 'O', 'O', 'O', 'B-cost_relative', 'O', 'O',
       'B-fromloc.city_name', 'B-fromloc.state_code', 'O',
       'B-toloc.city_name', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0'], dtype=object)

In [87]:
sortie_one_hot.shape

(3945, 41, 81)

### Modèle LSTM

In [153]:

config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = Dropout(config['dropout_rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=nbLabels)])
# NB : j'ai enlevé sparse car j'ai fait un vecteur one hot en sortie et sparse c'est pour quand c'est pas des one hot
print("compil ok")
print(model.summary())

config ok
create ok
compil ok


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 41, 256)        │       439,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 41, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 41, 81)         │        20,817 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,113 (1.76 MB)

 Trainable params: 460,113 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

None


### Entraînement du modèle LSTM
- x : entree de la forme nb_example x MAX_SEQ_SIZE x embedding_size
- y : sortie de la forme nb_example x MAX_SEQ_SIZE
- batch_size : nb d'échantillon à utiliser à chaque itération lors de l'entrainement
    - un batch_size + gd = accélère l'entrainement mais besoin de + de mémoire GPU
    - plus petit = ralentit l'entrainement mais meilleure convergence du modèle
- epochs : nb d'itération sur l'ens des données d'entrainement 
- validation_split : spécifie le fraction des données à utiliser comme données de validation

In [154]:
model.fit(x=entree, y=sortie_one_hot, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.2280 - accuracy_1: 0.0000e+00 - loss: 0.5729 - precision_1: 0.8572 - recall: 0.3716 - val_accuracy: 0.9637 - val_accuracy_1: 0.0000e+00 - val_loss: 0.1657 - val_precision_1: 0.9467 - val_recall: 0.8215
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.4015 - accuracy_1: 0.0000e+00 - loss: 0.1469 - precision_1: 0.9617 - recall: 0.8383 - val_accuracy: 0.9724 - val_accuracy_1: 0.0000e+00 - val_loss: 0.1214 - val_precision_1: 0.9590 - val_recall: 0.8646
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.4141 - accuracy_1: 0.0000e+00 - loss: 0.1029 - precision_1: 0.9686 - recall: 0.8817 - val_accuracy: 0.9783 - val_accuracy_1: 0.0000e+00 - val_loss: 0.1014 - val_precision_1: 0.9642 - val_recall: 0.8749
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.4153 - accuracy_1: 0.0000e+00 - loss: 0.0800 - precision_1: 0.9754 - recall: 0.8996 - val_accuracy: 0.9804 - val_accuracy_1: 0.000

### Prétraitement des données de test

In [155]:
with open("test_corpus", "r", encoding="utf-8") as file:
    data_test = file.readlines()
    
num_examples_test, MAX_SEQ_SIZE_TEST = count_examples_and_max_length(data_test)


vec_word_test, label_test_real = pad_sentences_from_file("test_corpus", MAX_SEQ_SIZE)

print("Nombre d'exemples :", num_examples_test) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE_TEST)

# Création du vecteur d'entrée 
test_data_input = np.zeros((num_examples_test, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word_test):
    for j, word in enumerate(sentence):
        if word:
            test_data_input[i, j] = get_embedding(word)
        else:
            test_data_input[i, j] = np.zeros(300)

Nombre d'exemples : 1033
Taille maximale de la phrase : 32


In [156]:
print(test_data_input.shape)
print(label_test_real.shape)
print(entree.shape)

(1033, 41, 300)
(1033, 41)
(3945, 41, 300)


In [157]:
label_test_real_one_hot = np.zeros((label_test_real.shape[0], label_test_real.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(label_test_real.shape[0]):
    for j in range(label_test_real.shape[1]):
        label = label_test_real[i, j]
        if label == '0':
            label_test_real_one_hot[i, j] = zero_vec
        else:
            label_test_real_one_hot[i, j] = label_one_hot_dict[label]

print(label_test_real_one_hot.shape)

(1033, 41, 81)


### Prédictions

In [158]:
test_predictions = model.predict(test_data_input)

print(test_predictions.shape)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
(1033, 41, 81)


In [159]:
predicted_labels_index = np.argmax(test_predictions, axis=-1)

predicted_labels = [[labels[idx] for idx in sample] for sample in predicted_labels_index]
predicted_labels = np.array(predicted_labels)
print(predicted_labels[10])

['O' 'O' 'O' 'O' 'O' 'B-fromloc.city_name' 'O' 'B-toloc.city_name' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']


In [160]:
print(label_test_real[10])

['O' 'O' 'O' 'O' 'O' 'B-fromloc.city_name' 'O' 'B-toloc.city_name' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']


In [162]:
test_loss, test_accuracy = model.evaluate(test_data_input, label_test_real_one_hot)

print("Loss on test data:", test_loss)
print("Accuracy on test data:", test_accuracy)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9854 - accuracy_1: 0.0000e+00 - loss: 0.0628 - precision_1: 0.9670 - recall: 0.9248
Loss on test data: 0.04819686338305473
Accuracy on test data: 0.9623639583587646
